## Systèmes de décision
### Projet - optimisation d'un emploi du temps
#### Partie 1 : modélisation, implémentation avec Gurobi, résolution et affichage de la surface de Pareto
<br>
<br>

<i>Sommaire :</i>
<br>
<br>
[1 - Chargement des données](#sect1)
<br>
<br>
[2 - Définition des constantes](#sect2)
<br>
<br>
[3 - Instanciation du modèle et des variables](#sect3)
<br>
<br>
[4 - Définition des fonctions objectifs](#sect4)
<br>
<br>
[5 - Définition des contraintes](#sect5)
<br>
<br>
[6 - Optimisation et vérification du respect des contraintes](#sect6)
<br>
<br>
[7 - Affichage des résultats et conclusion de la partie 1](#sect7)
<br>
<br>
<br>

### 1 - Chargement des données
<a id="sect1"> </a>

In [21]:
import gurobipy as gp # solver
from gurobipy import GRB
import numpy as np

In [13]:
from InstanceClass import Instance # custom class built to load and encode .json files (instances)

In [15]:
instance = Instance(path="data/toy_instance.json")
instance.build_instance()

In [16]:
instance.variables # check that everything works as expected

{'NP': 5,
 'NC': 3,
 'NA': 3,
 'H': 5,
 'GAIN': [20, 15, 15, 20, 10],
 'PENALTIES': array([[0., 0., 0., 3., 6.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 3.],
        [0., 0., 0., 3., 6.],
        [0., 0., 0., 0., 0.]]),
 'STAFF_QUALIFICATIONS': array([[1., 1., 1.],
        [1., 1., 0.],
        [0., 0., 1.]]),
 'COST_PROJECT': array([[1., 1., 1.],
        [1., 2., 0.],
        [1., 0., 2.],
        [0., 2., 1.],
        [0., 0., 2.]]),
 'CONGES': array([[0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.]])}

Quelques précisions sur la convention de nommage (également détaillée dans le compte-rendu) : 
<br>
<br>
$n_p$ : le nombre de projets de l'instance
<br>
$n_c$ : le nombre de collaborateurs de l'instance
<br>
$n_a$ :  le nombre de compétences présentes dans l'instance
<br>
$h$ :  l'horizon de temps du problème (en jours), début au jour 1
<br>
<br>
<br>

### 2 - Définition des constantes
<a id="sect2"> </a>
<br>
<br>
Nous pouvons définir certaines constantes du problème directement à partir de l'encodage de l'instance visible ci-dessus. Par convention, les constantes sont nommées en majuscule.

In [17]:
GAIN = instance.variables["GAIN"]
PENALTIES = instance.variables["PENALTIES"] # note : les lignes représentent les projets et les colonnes le temps en jours, les valeurs représentent ainsi la pénalité totale
COST_PROJECT = instance.variables["COST_PROJECT"] # cette variable représente le coût en compétences des projets
STAFF_QUALIFICATIONS = instance.variables["STAFF_QUALIFICATIONS"]
CONGES = instance.variables["CONGES"]
I = instance.variables["NC"]
J = instance.variables["H"]
L = instance.variables["NP"]
K = instance.variables["NA"]

<br>
On aura donc :
<br>
<i>i</i> : indice de sommation sur les collaborateurs
<br>
<i>j</i> : indice de sommation sur le temps
<br>
<i>k</i> : indice de sommation sur les compétences
<br>
<i>l</i> : indice de sommation sur les projets
<br>
<br>
Certaines quantités constantes du problème se dérivent à partir des quantités précédentes.
<br>
<br>

In [23]:
BENEFICES = np.array(
    [
        GAIN[time_step] - PENALTIES[time_step] for time_step in range(L)
    ]
) # bénéfice net que pourrait donner le projet i s'il était rendu au temps j (matrice)

In [24]:
MAX_COST = np.max(COST_PROJECT) # coût maximal en compétence du projet i

In [25]:
print(
    "Les collaborateurs sont numérotés de 1 à {}".format(I)
)
print(
    "Les jours sont numérotés de 1 à {}".format(J)
)
print(
    "Les compétences sont numérotés de 1 à {}".format(K)
)
print(
    "Les projets sont numérotés de 1 à {}".format(L)
)

Les collaborateurs sont numérotés de 1 à 3
Les jours sont numérotés de 1 à 5
Les compétences sont numérotés de 1 à 3
Les projets sont numérotés de 1 à 5


In [68]:
times = np.arange(J) + 1

<br>
<br>

### 3 - Instanciation du modèle et des variables
<a id="sect3"> </a>

Nous allons maintenant définir un tenseur des affectations qui contiendra 4 dimensions. Le tenseur sera noté $A$.
<br>
<br>
L'élément de tenseur $a_{i, j, k, l}$ vaudra 1 si et seulement si le collaborateur <i>i</i> travail le jour <i>j</i> selon la compétence <i>k</i> le projet <i>l</i> (et vaudra 0 sinon).
<br>
<br>
Certaines fonctions de sommation sur ce tenseur nous serons ensuite utiles pour exprimer les fonctions objectifs et contraintes.
<br>
<br>

In [27]:
def calcul_work(Affectation):
    """
    Entrée(s) :
        Affectation (type) : tenseur des affectations des collaborateurs par compétence, sur les projets et dans le temps
        
    Sortie(s) :
        work (numpy.array) : matrice contenant le travail des collaborateurs dans le temps selon les compétences (en lignes) et les projets (en colonnes)
    """
    work = [
        np.sum([Affectation[i][j] for i in range(3)] , axis=0) for j in range(np.shape(Affectation)[1])
    ]
    return np.array(work)

In [38]:
def calcul_temporel_work(work):
    """
    Entrée(s) :
        work (numpy.array) : matrice contenant le travail total des collaborateurs dans le temps selon les compétences (en lignes) et les projets (en colonnes)
        
    Sortie(s) :
        work_over_time (numpy.array) : vecteur contenant le travail total des collaborateurs selon le temps et les compétences, par projets (en lignes)
    """
    work_over_time = [
        [gp.quicksum(work[0:i])[j] for i in range(1, J+1)] for j in range(L)
    ]
    return np.array(work_over_time)

In [29]:
def calcul_work_final(work):
    """
    Entrée(s) :
        work (type) : tenseur des affectations des collaborateurs par compétence, sur les projets et dans le temps
        
    Sortie(s) :
        work_per_project (numpy.array) :
    """
    work_per_project = gp.quicksum(work)  
    return work_per_project

<br>
Nous pouvons désormais instancier le modèle et définir le tenseur $A$.
<br>
<br>
Note : la taille du tenseur $A$ est donc $n_c . h  . n_a. n_p$
<br>
<br>

In [26]:
model = gp.Model("model-part1")

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-29


In [30]:
Affectation = model.addMVar(
    shape=(I,J,L,K),
    vtype=GRB.BINARY,
    name="Affectation"
)

<br>
<br>
Nous définissons ensuite une matrice $E$ qui contient les projets (en lignes) et les jours (en colonnes).
<br>
<br>
L'élément de matrice $e_{i, j}$ vaut 1 si et seulement si le projet <i>i</i> est terminé le jour <i>j</i> (c'est-à-dire que tous les apports de travail par compétences ont été validés au jour <i>j</i>). Pour tous les jours pour lesquels le projet <i>i</i> n'est pas encore terminé ou pas encore commencé, $e_{i, j}$ vaut 0. Pour tous les jours ultérieurs au jour de terminaison du projet <i>i</i> (si pertinent), $e_{i, j}$ vaut 0 également.
<br>
<br>

In [31]:
Done_Project = model.addMVar(
    shape=(L, J),
    vtype=GRB.BINARY,
    name="Done_Project"
)

<br>
<br>
On peut définir la matrice $S$ analogue à $E$ mais qui contient les jours de début des projets, selon la même logique.
<br>
<br>

In [32]:
Begin_Project = model.addMVar(
    shape=(L, J),
    vtype=GRB.BINARY,
    name="Begin_Project"
) 

<br>
<br>

### 4 - Définition des fonctions objectifs
<a id="sect4"> </a>

Pour simplifier la définition des fonctions objectifs (et anticiper le calcul prochain de la surface de Pareto du problème), on définit une fonction <i>set_objectives</i> qui va nous servir à intégrer, à volonté, certaines fonctions objectifs au problème.
<br>
<br>
<i>Remarque</i> : on souhaite à terme intégrer toutes les fonctions objectifs au problème, bien entendu !
<br>
<br>

In [56]:
def set_objectives(
    model,
    include_gain_obj: bool,
    include_nb_proj_obj: bool,
    include_proj_time_obj: bool
):
    """
        Entrée(s) :
            model:
            include_gain_obj: bool,
            include_nb_proj_obj: bool,
            include_proj_time_obj: bool
        
        Sortie(s) :
            
    """
    z_fo1 = gp.quicksum(gp.quicksum(Done_Project*BENEFICES))
    z_fo2 = model.addVar(vtype=GRB.INTEGER, name="Max_Projet")
    z_fo3 = model.addVar(vtype=GRB.INTEGER, name="Duree_Max_Projet")
    
    if include_gain_obj:
        model.setObjective(
            z_fo1,
            GRB.MAXIMIZE
        )
        
    if include_nb_proj_obj:
        model.setObjective(
            z_fo2,
            GRB.MINIMIZE
        )

    if include_proj_time_obj:
        model.setObjective(
           z_fo3,
           GRB.MINIMIZE
       )
    return z_fo1, z_fo2, z_fo3

In [57]:
 z_fo1, z_fo2, z_fo3 = set_objectives(
    model, 
    True,
    False,
    False
)

<br>
<br>

### 5 - Définition des contraintes
<a id="sect5"> </a>

<br>
<br>
Avant de passer à la définition des contraintes du problème, nous allons instancier deux quantités annexes, les "travaux partiels", définis à partir des fonctions déclarées au début de la section <a>3 - Instanciation du modèle et des variables</a>.
<br>
<br>

In [67]:
final_work = calcul_work_final(calcul_work(Affectation)) # utile pour vérifier qu'un projet est terminé
temp_work = calcul_temporel_work(calcul_work(Affectation))
# somme cumulée dans le temps du travail effectué par projet et par compétence

<br>
<br>
On va d'abord définir les contraintes "de base" du problème.
<br>
<br>

In [73]:
def base_constraints():
    """
    Desc.
    """
    model.addConstrs(
        (Affectation[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] for i in range(I) for j in range(J) for l in range(L) for k in range(K)),
        name="Qualifications_Constraint"
    )
    model.addConstrs(
        (gp.quicksum(Affectation[i][j] @ np.transpose(np.array([1]*K))) <= 1-CONGES[i][j] for i in range(I) for j in range(J) for l in range(L)),
        name="Day_off_Constraint"
    ) 
    model.addConstrs(
        (temp_work[i][j] >= COST_PROJECT[i]*Done_Project[i][j] for i in range(L) for j in range(J)),
        name="Done"
    ) 
    model.addConstrs(
        (gp.quicksum(Done_Project[i]) <= 1 for i in range(L)),
        name="Only_Done_One_Time"
    ) 
    model.addConstrs(
        (gp.quicksum(Begin_Project[i])<= 1 for i in range(L)),
        name="Only_Begin_One_Time"
    ) 
    model.addConstrs(
        (gp.quicksum(Begin_Project[i]) == gp.quicksum(Done_Project[i]) for i in range(L)),
        name="Begin_Done"
    ) 
    model.addConstrs(
        (gp.quicksum((Done_Project[i]-Begin_Project[i])*u)+1<=z_fo3 for i in range(L)),
        name="Duration"
    ) 
    model.addConstrs(
        (temp_work[i][j][k]/MAX_COST <= gp.quicksum(Begin_Project[i][0:(j+1)]) for i in range(L) for j in range(J) for k in range(K)),
        name="Cant_begin_without_work"
    ) 


In [74]:
base_constraints()

<br>
<br>
On va ensuite définir les contraintes additionnelles provenant de la linéarisation des fonctions objectifs initiales.
<br>
<br>

In [75]:
def fo2_constraints(epsilon: float = 0.0001):
    """
    Desc.
    """
    delta = model.addMVar(
        shape=(L, I),
        vtype=GRB.INTEGER,
        name="deltaFromC2")
    
    model.addConstrs(
        (gp.quicksum(delta)[i] <= z_fo2 for i in range(I)),
        name="C2.1"
    ) 
    model.addConstrs(
        ((J*(1-delta[l][i]))<=gp.quicksum(gp.quicksum(Affectation[i])[l]) for i in range(I) for l in range(L)),
        name="C2.2"
    )
    model.addConstrs(
        (gp.quicksum(gp.quicksum(Affectation[i])[l])+epsilon<=(J*delta[l][i]) for i in range(I) for l in range(L)),
        name="C2.3"
    ) 

In [76]:
fo2_constraints()

### 6 - Optimisation et vérification du respect des contraintes
<a id="sect6"> </a>

In [77]:
model.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (win64)

CPU model: Intel(R) Core(TM) i3-6006U CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 2189 rows, 352 columns and 14129 nonzeros
Model fingerprint: 0x76109fee
Variable types: 0 continuous, 352 integer (275 binary)
Coefficient statistics:
  Matrix range     [5e-01, 5e+00]
  Objective range  [9e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-04, 5e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 2116 rows and 236 columns
Presolve time: 0.07s
Presolved: 73 rows, 116 columns, 512 nonzeros
Variable types: 0 continuous, 116 integer (116 binary)

Root relaxation: objective 8.000000e+01, 49 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   80.0

#### Vérification des contraintes
<br>
<br>

In [78]:
Affectation_solution = Affectation.X
Done_Project_solution = Done_Project.X
Begin_Project_solution = Begin_Project.X
NB_Project_solution = z_fo2.X
Duration_solution = z_fo3.X

In [79]:
cumul_work = np.cumsum(np.sum(Affectation_solution,axis=0),axis=0)

In [80]:
# Qualifications
for i in range(I):
    for j in range(J):
        for l in range(L):
            for k in range(K):
                assert Affectation_solution[i,j,l,k] <= STAFF_QUALIFICATIONS[i,k] 

In [81]:
# One day One work
for i in range(I):
    for j in range(J):
        assert np.sum(Affectation_solution[i][j])<= 1- CONGES[i][j]

In [82]:
# Project Done one time
assert (np.sum(Done_Project_solution,axis=1)<=1).all()

In [83]:
# Assert Project done
for l in range(L):
    if np.sum(Done_Project_solution[l])==1:
        assert(
                (cumul_work[
                    np.where(Done_Project_solution[l] == 1)[0][0]
                ][l] >= COST_PROJECT[l]).all()
            )

In [84]:
# Project Done & Begin
assert((
np.sum(Done_Project_solution,axis=1)==np.sum(Begin_Project_solution,axis=1)
).all())

In [85]:
# Project are begin before
assert((
Done_Project_solution<=np.cumsum(Begin_Project_solution,axis=1)
).all())

In [86]:
# Assert Duration Project
for l in range(L):
    if np.sum(Done_Project_solution[l])==1:
        assert(
            (np.where(Done_Project_solution[l] == 1)[0][0] - np.where(Begin_Project_solution[l] == 1)[0][0])+1 <= Duration_solution
            )

In [87]:
# Assert Nb project worked
for i in range(i):
    assert np.sum(np.sum(np.sum(Affectation_solution,axis=1)[i],axis=1)>=1)<=NB_Project_solution

In [88]:
# Working imply Beginned
for l in range(L):
    if np.sum(Done_Project_solution[l])==1:
        assert np.where(np.sum(cumul_work,axis=2).transpose()[l]>=1)[0][0] >= np.where(Begin_Project_solution[l]>=1)[0][0]

<br>

### 7 - Affichage des résultats et conclusion de la partie 1
<a id="sect7"> </a>

Chargement des résultats de l'optimisation.
<br>

In [22]:
instance.load_solution(model,Affectation,Done_Project,Begin_Project,z_fo2,z_fo3)
instance.save_solution(path="medium_instance/")
instance.print_kpi()

The project Job1 is selected in the solution and done during the day 26 and beginned during the day 1.
The project Job2 is selected in the solution and done during the day 22 and beginned during the day 1.
The project Job3 is selected in the solution and done during the day 15 and beginned during the day 1.
The project Job4 is selected in the solution and done during the day 17 and beginned during the day 1.
The project Job5 is selected in the solution and done during the day 14 and beginned during the day 1.
The project Job6 is selected in the solution and done during the day 33 and beginned during the day 1.
The project Job7 isn't selected in the solution.
The project Job8 isn't selected in the solution.
The project Job9 isn't selected in the solution.
The project Job10 isn't selected in the solution.
The project Job11 isn't selected in the solution.
The project Job12 is selected in the solution and done during the day 29 and beginned during the day 1.
The project Job13 isn't selecte

#### Résultat de l'optimisation de la fonction objectif 1 (gain total) sur le jeu "Toy"
<br>
<br>
FO1 (maximisation du gain total) : 0.1875 (1 - 65/80)
<br>
Attention ! On a ici calculé $1 - val_{opt}(FO1)$. Une valeur de 0 voudrait dire que 100% des gains possibles ont été obtenus, pénalités déduites.
<br>
<br>
FO2 (minimisation du nombre de projets abordés par le collaborateur affecté sur le plus grand nombre de projets) : 0.60 (3/5)
<br>
Une valeur de 0 voudrait dire que le collaborateur le plus occupé a participé à tous les projets.
<br>
<br>
FO3 (minimisation du temps du projet qui a pris le plus de temps) : 0.60 (3/5)
<br>
Une valeur de 0 voudrait dire que le projet ayant duré le plus longtemps a duré $h$ jours.
<br>
<br>

#### Résultat de l'optimisation de la fonction objectif 1 (gain total) sur le jeu "Medium"
<br>
<br>
FO1 (maximisation du gain total) : 0.345 (176/510)
<br>
<br>
FO2 (minimisation du nombre de projets abordés par le collaborateur affecté sur le plus grand nombre de projets) : 0.4 (6/15)
<br>
<br>
FO3 (minimisation du temps du projet qui a pris le plus de temps) : 0.41 (9/22)
<br>
<br>

#### Résultat de l'optimisation de la fonction objectif 1 (gain total) sur le jeu "Large"
<br>
<br>
FO1 (maximisation du gain total) : 0.248 (253/1020)
<br>
<br>
FO2 (minimisation du nombre de projets abordés par le collaborateur affecté sur le plus grand nombre de projets) : 0.32 (8/25)
<br>
<br>
FO3 (minimisation du temps du projet qui a pris le plus de temps) : 0.334 (12/36) 
<br>
<br>

#### Affichage de la surface de Pareto pour le jeu "Toy"
<br>
<br>
Selon l'axe 1 (FO1), l'optimal est trouvé pour une valeur de 0.1875 et correspond au couple (FO2, FO3) = (0.60, 0.60).
<br>
Selon l'axe 2 (FO2), l'optimal est trouvé pour une valeur de 0 et correspond au couple (FO1, FO3) = (1, 0). Pas du tout de travail (trivial).
<br>
Selon l'axe 3 (FO3), l'optimal est trouvé pour une valeur de 0 et correspond au couple (FO1, FO2) = (1, 0). Pas du tout de travail (trivial).
<br>
<br>
Nous en concluons que les coordonnées de la grande diagonale du cube de Pareto sont :
<br>
<br>

In [89]:
p1 = (
    1.0,
    0.6,
    0.6
)
p2 = (
    0.1875,
    0.0,
    0.0
)

In [92]:
# imports pour l'affichage
import plotly.graph_objects as go
import numpy as np

In [91]:
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 15.2/15.2 MB 3.8 MB/s eta 0:00:00


In [100]:
fig = go.Figure(data=[
    go.Mesh3d(
        # 8 vertices of a cube
        x=[p1[0], p1[0], p2[0], p2[0], p1[0], p1[0], p2[0], p2[0]],
        y=[p1[1], p2[1], p2[1], p1[1], p1[1], p2[1], p2[1], p1[1]],
        z=[p1[2], p1[2], p1[2], p1[2], p2[2], p2[2], p2[2], p2[2]],

        # i, j and k give the vertices of triangles
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
        name='y',
        opacity=0.50
    )
])
fig.update_layout(
    scene = dict(
                xaxis = dict(nticks=10, range=[0,1],),
                yaxis = dict(nticks=10, range=[0,1],),
                zaxis = dict(nticks=10, range=[0,1],),
                xaxis_title='1 - Gain',
                yaxis_title='# projets du collab. le + occupé',
                zaxis_title='Durée du projet le plus long',
    ),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))
fig.update_layout(title_text="Cube de Pareto (axes normalisé sur [0, 1])")
fig.show()